# Network-based stress testing 
Network-based stress testing using the Network Valuation in financial systems (NEVA) framework.
https://github.com/marcobardoscia/neva  
https://github.com/marcobardoscia/neva/blob/master/README.md  
For [Bardoscia's slides](https://paolobaruccadb.files.wordpress.com/2017/01/bdf-tutorials-network-based-stress-testing.pdf) on the theory, see [4].

## Overview

Channels of financial contagion that cause systemic risk include solvency, distress & liquidity contagion.

Define a simple financial system composed of institutions with balance sheets:
* external assets
* external liabilities
* interbank liabilities (matrix)

Mechanisms of financial contagion:
* Furfine
* Eisenberg-Noe 
* Rogers-Veraart
* DebtRank
* Endogenous DebtRank

|Model	|Valuation time	|Network propagation	|Default losses	|Endogenous Recovery	|
|:------ |:----- |:----- |:----- |:----- |
|Merton	|Ex-ante	|None	|None	|None	|
|Eisenberg Noe	|Ex-post	|Local	|None	|Full	|
|Rogers Veraart	|Ex-post	|Local	|Present	|Full	|
|Linear DebtRank|Ex-ante	|Local	|Present	|None	|
|Fischer Model	|Ex-ante	|Global	|None	|Full	|
|NEVA	|Ex-ante	|Local	|Present	|Full	|


The Network Valuation in financial systems (NEVA) framework 
* asks the question: "What is the net value of a financial institution in a network?". 
* takes into account at the same time interdependencies (as in Furfine 1999, Eisenberg and Noe 2001) and uncertainty (as in Merton 1974) with local exchange of information.

### Install package from github - first time only!

This only needs to be done the first time that you run the notebook after cloning the notebook into your own account. 

Clone git repository to Azure. Output should say:  
    `Cloning into 'neva'...  
    Checking out files: 100% (23/23), done.`

In [32]:
!git clone git://github.com/marcobardoscia/neva.git

Cloning into 'neva'...
remote: Counting objects: 32, done.
remote: Total 32 (delta 0), reused 0 (delta 0), pack-reused 32
Receiving objects: 100% (32/32), 25.06 KiB | 0 bytes/s, done.
Resolving deltas: 100% (4/4), done.
Checking connectivity... done.
Checking out files: 100% (23/23), done.


Change directory into the newly-created `neva` folder, to run the script to install the `neva` package, ready for use by the python code in the notebook.

In [33]:
%cd neva
!python setup.py install
%cd /home/nbuser/library   # Move back up to /home/nbuser/library

/home/nbuser/library/neva/data/neva
running install
running bdist_egg
running egg_info
creating neva.egg-info
writing neva.egg-info/PKG-INFO
writing dependency_links to neva.egg-info/dependency_links.txt
writing top-level names to neva.egg-info/top_level.txt
writing manifest file 'neva.egg-info/SOURCES.txt'
reading manifest file 'neva.egg-info/SOURCES.txt'
reading manifest template 'MANIFEST.in'
writing manifest file 'neva.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build
creating build/lib
creating build/lib/neva
copying neva/ibeval_lender.py -> build/lib/neva
copying neva/exteval.py -> build/lib/neva
copying neva/bankingsystem.py -> build/lib/neva
copying neva/utils.py -> build/lib/neva
copying neva/adjust.py -> build/lib/neva
copying neva/__init__.py -> build/lib/neva
copying neva/bank.py -> build/lib/neva
copying neva/parse.py -> build/lib/neva
copying neva/gbm.py -> build/lib/neva
copying neva/ibeval.p

In [16]:
# Append the package to the path
# import sys
# sys.path.append("/home/nbuser/library/neva/data") # Still can't find the data. Have to cd there!
# sys.path.append('/home/nbuser/anaconda3_501/lib/python3.6/site-packages/neva-0.1-py3.6.egg')

In [21]:
# Change directory to where the neva data files are
%cd /home/nbuser/library/neva/data

/home/nbuser/library/neva/data


## Simplest example - Eisenberg and Noe
In the simplest example we will run the Eisenberg and Noe algorithm without 
any shocks to equities.

In [5]:
import neva

In [6]:
help(neva)

Help on package neva:

NAME
    neva

PACKAGE CONTENTS
    adjust
    bank
    bankingsystem
    exteval
    gbm
    ibeval
    ibeval_lender
    parse
    utils

FILE
    /home/nbuser/anaconda3_501/lib/python3.6/site-packages/neva-0.1-py3.6.egg/neva/__init__.py




See:
* [balance_sheets.csv](https://github.com/marcobardoscia/neva/blob/master/data/balance_sheets.csv),  
* [exposures_table.csv](https://github.com/marcobardoscia/neva/blob/master/data/exposures_table.csv)

In [22]:
# parsing data
bsys, params = neva.parse_csv('balance_sheets.csv', 'exposures_table.csv')

In [23]:
# running Eisenberg and Noe (without any shock to equities)
equity_shock = [0.0 for _ in bsys]
neva.shock_and_solve(bsys, equity_shock, method='eisenberg_noe', solve_assets=False)

In [24]:
# reading final equities
equity_final = bsys.history[-1]

In [25]:
equity_final

[1.0, 1.0, 1.0]

In [26]:
# computing payment vectors
pay_vec = [bnk.ibliabtot if bnk.equity >= 0 else
           max(bnk.equity + bnk.ibliabtot, 0.0) for bnk in bsys]

In [12]:
pay_vec

[0.5, 0.5, 0.5]

## A more complex example
Here we run an analysis similar to that run in [2].

In [27]:
# Change directory to where the neva data files are
%cd /home/nbuser/library/neva/data

/home/nbuser/library/neva/data


See:
* [balance_sheets.csv](https://github.com/marcobardoscia/neva/blob/master/data/balance_sheets.csv),  
* [exposures_table.csv](https://github.com/marcobardoscia/neva/blob/master/data/exposures_table.csv)

In [14]:
import neva

# parsing data
bsys, params = neva.parse_csv('balance_sheets.csv', 'exposures_table.csv')

# Geometric Browianian Motion on external assets, whose volatility is
# estimated via the volatility of equities.
sigma_equity = [float(params[bnk]['sigma_equity']) for bnk in params]
bsys = neva.BankingSystemGBMse.with_sigma_equity(bsys, sigma_equity)
    
# storing initial equity
equity_start = bsys.get_equity()

# shocks to initial equity: 50%
equity_delta = equity_start[:]
equity_delta = [e * 0.5 for e in equity_start]

# running ex-ante Black and Cox, as in [2] 
# with recovery rate equal to 60%
recovery_rate = [0.6 for _ in bsys] 
neva.shock_and_solve(bsys, equity_delta, 'exante_en_blackcox_gbm', 
                     solve_assets=False, recovery_rate=recovery_rate)

# reading equities after one round and after all rounds   
equity_direct = bsys.history[1]
equity_final = bsys.history[-1]

# Further reading

[1] Barucca, Paolo, Marco Bardoscia, Fabio Caccioli, Marco D’Errico, Gabriele Visentin, Stefano Battiston, and Guido Caldarelli. “Network Valuation in Financial Systems.” SSRN Scholarly Paper. Rochester, NY: Social Science Research Network, June 14, 2016. https://papers.ssrn.com/abstract=2795583.  
[2] Bardoscia, Marco, Paolo Barucca, Adam Brinley Codd, and John Hill. “The Decline of Solvency Contagion Risk.” SSRN Scholarly Paper. Rochester, NY: Social Science Research Network, June 30, 2017. https://papers.ssrn.com/abstract=2996689.    
[3] Bardoscia, Marco. Neva: Network Valuation in Financial Systems. Python, 2017. https://github.com/marcobardoscia/neva.  
[4] Barucca, Paolo. “BDF Tutorials: Network Based Stress Testing.” Asymptotic Freedom (blog), January 6, 2017. https://paolobarucca.com/courses/bdf-tutorials-network-based-stress-testing/.  
[5] ———. “Network Valuation in Financial Systems.” 2017. https://icspconference.files.wordpress.com/2016/12/28_barucca.pdf.  
[6] rayohauno. Nonlinear_debt_rank: This Is (the Essential) Part of the Code Used to Run the Simulations for "Distress Propagation in Complex Networks: The Case of Non-Linear DebtRank", M. Bardoscia, F.. Python, 2016. https://github.com/rayohauno/nonlinear_debt_rank.  

Barucca, Paolo. “BDF Tutorials: Network Based Stress Testing.” 2017. https://paolobaruccadb.files.wordpress.com/2017/01/bdf-tutorials-network-based-stress-testing.pdf.  
———. “BDF Tutorials: Network Based Stress Testing.” Asymptotic Freedom (blog), January 6, 2017. https://paolobarucca.com/courses/bdf-tutorials-network-based-stress-testing/.  
———. Debtrank: Matlab Implementation of the Algorithm Described in Battiston et Al. 2012. Matlab, 2017. https://github.com/paolobarucca/debtrank.  
———. Eisenbergnoe: Matlab Implementation of the Algorithm Described in Eisenberg and Noe 2001. Matlab, 2017. https://github.com/paolobarucca/eisenbergnoe.  
———. Endogenousdebtrank: Matlab Implementation of the Algorithm Described in Barucca et Al. 2016. Matlab, 2017. https://github.com/paolobarucca/endogenousdebtrank.  
———. Furfine: Matlab Implementation of the Algorithm Described in Furfine 1999. Matlab, 2017. https://github.com/paolobarucca/furfine.  
———. Neva: Matlab Implementation of the General Framework Described in Barucca et Al. 2016. Matlab, 2017. https://github.com/paolobarucca/neva.  
———. Rogersveraart: Matlab Implementation of the Algorithm Described in Rogers and Veraart 2013. Matlab, 2017. https://github.com/paolobarucca/rogersveraart.  
“Big Data Finance 2017: Network-Based Stress Testing - Exercises.” Accessed July 18, 2017. https://paolobaruccadb.files.wordpress.com/2017/01/exercises.pdf.  